# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
keras = tf.keras
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler
import plotly.graph_objects as go
from math import sqrt
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Importing Data

In [2]:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

# Data Pre-Processing

In [3]:
DAYS_TO_PREDICT = int(input('Enter Number Of Days You Want to Predict : '))  # last days to predict  
death_df = death_df.groupby("Country/Region").sum().reset_index()
Country_death =  death_df['Country/Region'].values
death_df = death_df.drop(['Lat','Long','Country/Region'], axis=1)
death_df = death_df[death_df.columns].T
death_df.columns = Country_death
compare_death_df=death_df.iloc[-DAYS_TO_PREDICT-1:,:]                 # last days to predict
death_df= death_df.iloc[:-DAYS_TO_PREDICT,:]                        # last days to predict

Date_index = death_df.index
death_df = death_df.diff().fillna(0)
compare_death_df = compare_death_df.diff().dropna(0)
death_df = death_df.abs()
death_df

Enter Number Of Days You Want to Predict : 5


,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burma,Burundi,Cabo Verde,Cambodia,Cameroon,Canada,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo (Brazzaville),...,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Slovakia,Slovenia,Somalia,South Africa,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan*,Tajikistan,Tanzania,Thailand,Timor-Leste,Togo,Trinidad and Tobago,Tunisia,Turkey,US,Uganda,Ukraine,United Arab Emirates,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
1/22/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/23/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/24/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/25/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/26/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6/18/20,42.0,1.0,12.0,0.0,1.0,0.0,35.0,7.0,0.0,1.0,6.0,0.0,6.0,38.0,0.0,7.0,8.0,0.0,2.0,0.0,18.0,0.0,0.0,1238.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,1.0,0.0,226.0,0.0,68.0,2.0,0.0,...,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0,1.0,0.0,0.0,0.0,1.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,715.0,0.0,23.0,3.0,135.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
6/19/20,2.0,3.0,14.0,0.0,0.0,0.0,31.0,10.0,0.0,0.0,4.0,0.0,2.0,45.0,0.0,6.0,12.0,0.0,0.0,0.0,18.0,1.0,0.0,1206.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,6.0,47.0,0.0,0.0,252.0,0.0,91.0,0.0,0.0,...,3.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,94.0,2.0,1179.0,0.0,19.0,1.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,692.0,0.0,19.0,2.0,173.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0
6/20/20,21.0,1.0,12.0,0.0,1.0,0.0,13.0,13.0,0.0,0.0,5.0,0.0,3.0,37.0,0.0,6.0,1.0,0.0,0.0,0.0,25.0,0.0,0.0,1022.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,19.0,58.0,0.0,0.0,202.0,1.0,102.0,0.0,0.0,...,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,0.0,7.0,0.0,15.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,596.0,0.0,9.0,1.0,128.0,1.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0
6/21/20,12.0,1.0,8.0,0.0,0.0,0.0,19.0,18.0,0.0,2.0,6.0,0.0,3.0,39.0,0.0,3.0,0.0,0.0,2.0,0.0,33.0,0.0,0.0,615.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,184.0,0

# Enter countries you want to forecast

In [4]:
i = int(input('Number of countries you want to consider : '))
countries = [input('Enter country You Want to forecast : ') for i in range(i)]

Number of countries you want to consider : 3
Enter country You Want to forecast : Germany
Enter country You Want to forecast : Italy
Enter country You Want to forecast : China


In [5]:
countries

['Germany', 'Italy', 'China']

In [6]:
death_df = death_df.loc[:,countries]   
compare_death_df = compare_death_df.loc[:,countries]  # last days to predict 

# Scalling the data

In [7]:
sc = RobustScaler()
death_df_scaled = sc.fit_transform(death_df)
death_df_scaled = pd.DataFrame(death_df_scaled)

death_df_scaled

,0,1,2
0,-0.173913,-0.267470,-0.038462
1,-0.173913,-0.267470,0.000000
2,-0.173913,-0.267470,0.269231
3,-0.173913,-0.267470,0.576923
4,-0.173913,-0.267470,0.500000
...,...,...,...
148,0.086957,-0.108434,-0.038462
149,-0.043478,-0.154217,-0.038462
150,-0.086957,-0.149398,0.000000
151,-0.173913,-0.209639,-0.038462


# Enter the number of days you want to predict

In [8]:
#DAYS_TO_PREDICT = int(input('Enter Number Of Days You Want to Predict : '))
predict_death_index = pd.date_range(start=Date_index[-1],periods=DAYS_TO_PREDICT + 1,closed='right')
predict_death_index = predict_death_index.strftime('%Y/%m/%d')

# Creating class for building model

In [9]:
class Corona_death_model():
    def create_dataset(self, dataset, time_steps=1, column = 0):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_steps-1):
          a = dataset[i:(i+time_steps), column]
          dataX.append(a)
          dataY.append(dataset[i + time_steps, column])
        return np.array(dataX), np.array(dataY)

    def Model(self,dframe,column):
        df = dframe.values
        df = df.astype('float32')
        
        time_steps = 1
        trainX, trainY = self.create_dataset(df, time_steps, column)
        trainX = np.reshape(trainX, (trainX.shape[0],1,trainX.shape[1]))
        
        
        keras.backend.clear_session()
        tf.random.set_seed(42)
        np.random.seed(42)
        death_model = keras.models.Sequential([
            keras.layers.InputLayer(input_shape=[None, 1]),
            keras.layers.Conv1D(filters=16, kernel_size=2,
                      strides=1, padding="causal",
                      activation="relu",
                      ),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(1,activation = 'linear',kernel_initializer = 'he_normal'),
            keras.layers.Lambda(lambda x: x * 200)
             ])
               
        optimizer = keras.optimizers.Adadelta(lr = 0.001)
        
        death_model.compile(loss=['mae'],optimizer=optimizer)   
                
        death_model.fit(trainX, trainY, epochs=100,
                          batch_size = 10,verbose = 0)

        death_test_seq  = trainX[-1:]
        death_test_seq = death_test_seq
        death_test_seq = death_test_seq.reshape(len(death_test_seq), time_steps, 1)
        death_preds = []
        
        for _ in range (DAYS_TO_PREDICT):
            death_pred = death_model.predict(death_test_seq)
            death_preds.append(death_pred)
            death_new_seq = death_pred
            death_test_seq = death_new_seq
            death_test_seq = death_test_seq.reshape(len(death_test_seq ), time_steps, 1)

        

        return death_preds

# Calling the class and predicting the cases for every individual country

In [10]:
NN = Corona_death_model()
death_predictions = []
for i in range(len(countries)):
    result = NN.Model(death_df_scaled,i) 
    death_predictions.append(result)

In [11]:
death_predictions = (np.array(death_predictions).transpose())
death_predictions = death_predictions.reshape(((DAYS_TO_PREDICT),len(countries)))
death_predictions = sc.inverse_transform(death_predictions)
death_predictions = pd.DataFrame(death_predictions)
death_predictions = death_predictions.abs()
death_predictions['Dates'] = predict_death_index
death_predictions.set_index('Dates', inplace = True)
death_predictions.columns = countries
death_predictions.head()

,Germany,Italy,China
Dates,,,
2020/06/23,5.966923,39.751060,0.021283
2020/06/24,3.720810,28.530426,0.091756
2020/06/25,2.258086,18.695671,0.118201
2020/06/26,1.299591,9.992554,0.128132
2020/06/27,0.668849,4.530312,0.131858


# Visualizing The Death Cases For A Country As Per The User's Choice

### List of Infected Countries

In [14]:
Infected_Countries = pd.DataFrame(countries)
Infected_Countries.columns = ['Countries']
pd.set_option("max_rows", None)
Infected_Countries

,Countries
0,Germany
1,Italy
2,China


### Select The Country

In [15]:
Country = input('Enter the name of the country from above list you would like to see the daily cases till now : ')

Enter the name of the country from above list you would like to see the daily cases till now : Italy


## Till Date Daily Cases of Death due to coronavirus

In [16]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Death Daily Cases','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},
                                    xaxis_title="Dates",
                                    yaxis_title="Number Of Cases")
fig.add_trace(go.Scatter(x=death_df.index, 
                         y=death_df[Country],
                         mode='lines+markers',
                         line=dict(color='red', width=2)))

## Predictions of death cases due to coronavirus

In [17]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Death Cases Predictions','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},xaxis_title="Dates",yaxis_title="Number Of Cases")                           
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=death_predictions.index, 
                         y=death_predictions[Country],
                         mode='lines+markers',
                         line=dict(color='turquoise', width=2)))

# Root mean square error between last days predictions(Only for comment last days to predict)

In [18]:
compare_death_df

,Germany,Italy,China
6/23/20,15.0,18.0,1.0
6/24/20,14.0,-31.0,0.0
6/25/20,12.0,34.0,1.0
6/26/20,25.0,30.0,0.0
6/27/20,3.0,8.0,0.0


In [19]:
death_predictions

,Germany,Italy,China
Dates,,,
2020/06/23,5.966923,39.751060,0.021283
2020/06/24,3.720810,28.530426,0.091756
2020/06/25,2.258086,18.695671,0.118201
2020/06/26,1.299591,9.992554,0.128132
2020/06/27,0.668849,4.530312,0.131858


In [20]:
for country in countries:
  predictVals = death_predictions[country]
  realVals = compare_death_df[country]
  rmse = sqrt(mean_squared_error(realVals,predictVals))
  print('RMSE for {} : '.format(country),rmse)
  

RMSE for Germany :  13.033086863212196
RMSE for Italy :  30.540266480101494
RMSE for China :  0.5962682783556845
